### 4.1 FewShotPromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# AI에게 줄 답변 예제들
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


In [ ]:
# Before I give examples to Language Model(LM)
chat.predict("What do you know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.

France is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of over 67 million people. The official language is French, and the currency is the Euro.

France is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, countryside, and coastline. The country is also known for its wine and cheese production, fashion industry, and art and literature.

France has a long history of political and cultural influence, including the French Revolution, the Napoleonic Wars, and the establishment of the French colonial empire. Today, France is a member of the United Nations, NATO, and the European Union, among other international organizations.

'France is a country located in Western Europe. It is known for its rich history, culture, and cuisine. The capital city is Paris, which is famous for landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral.\n\nFrance is the largest country in the European Union by land area and the third-largest in Europe overall. It has a population of over 67 million people. The official language is French, and the currency is the Euro.\n\nFrance is a popular tourist destination, attracting millions of visitors each year to its beautiful cities, countryside, and coastline. The country is also known for its wine and cheese production, fashion industry, and art and literature.\n\nFrance has a long history of political and cultural influence, including the French Revolution, the Napoleonic Wars, and the establishment of the French colonial empire. Today, France is a member of the United Nations, NATO, and the European Union, among other international organizations.'

In [ ]:
# {}안의 변수 이름은 예제와 같게 설정
example_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you konw about {country}?", # 앞의 예제를 보고나서 AI가 답변해야 할 질문
    input_variables=["country"]
)

chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AI: 
        Here is what I know:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

### 4.2 FewShotChatMessagePromptTemplate

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# AI에게 줄 답변 예제들
examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 형식 지정
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

# 형식에 맞게 대화형으로 답변
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 초기 설정, 예제와 함께 새로운 질문
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country": "Thailand"
})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')